In [216]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension ("tabulator", template="material", sizing_mode="stretch_width")
import holoviews as hv
import hvplot.pandas
import datetime as dt

In [311]:
# loading data and showing it as a table on top of the dashboard
#https://docs.google.com/spreadsheets/d/13ikD5WpjmapBlKY4j2dTrwsZ-du3pdfpHhjIYwugcxM/edit?usp=sharing

sheet_id="13ikD5WpjmapBlKY4j2dTrwsZ-du3pdfpHhjIYwugcxM"    
df_google_sheet= pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")    #add this /export?format=csv
useful_data=df_google_sheet
df_google_sheet=df_google_sheet.interactive()
df_google_sheet1=df_google_sheet.hvplot.table( sortable=True, selectable=True)  
#df_google_sheet1


In [312]:
#table 2
#count the number of surveys carried out by each enumerator
enumrator_survey_count=useful_data['Enumerator_Name'].value_counts().rename_axis("Enumerator Name").reset_index(name="count")
#pd.DataFrame(count,columns=["Name","Count"])
enumrator_survey_count

,Enumerator Name,count
0,Neelam shah,14
1,KaleemUllah Quetta,11
2,Aftab khan dotani,11
3,wali khan,11
4,sanaullah quetta,11
5,Abdul Rahim,10
6,Akhtar Muhammad,8
7,Sameera Aslam Quetta,8
8,ubaid ur rehman,7
9,Yahya khan,6


In [313]:
# table 2 into tabuler

filter_table = pn.widgets.Tabulator(
    enumrator_survey_count, pagination='remote', layout='fit_columns', page_size=10, sizing_mode='stretch_width'
    #header_filters=filters , editors=editors
)

In [314]:
#table 3 
#adding date filter
#first is to make the widget
datetime_range_slider = pn.widgets.DatetimeRangeSlider(
    name='Datetime Range Slider',
    start=dt.datetime(2022, 12, 1), end=dt.datetime(2022, 12, 31),
    value=(dt.datetime(2022, 12, 1), dt.datetime(2022, 12, 31)),
    step=10000
)

datetime_range_slider


BokehModel(combine_events=True, render_bundle={'docs_json': {'966fc592-6ef6-4518-986e-33247934d412': {'defs': …

In [315]:
#make a dataframe that has filtered data
date=useful_data#[["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"A14","Survey_Day"]]
date["Survey_Day"]=pd.to_datetime(date["Survey_Day"]) #convert format from object to date_type
# filter_table2 = pn.widgets.Tabulator(
#     merge_total, pagination='remote', layout='fit_columns', page_size=10, sizing_mode='stretch_width'
#     #header_filters=filters , editors=editors
# )

In [316]:
#date_filtered_table_survey_count_bif=date[(date["Survey_Day"]>=datetime_range_slider.value[0]) & (date["Survey_Day"]<=datetime_range_slider.value[1])]
#date_filtered_table_survey_count_bif=date_filtered_table_survey_count_bif.interactive()

In [349]:
# table 3 
#get the bifercaton of male and female from column a14 ,1 means male, 2 means female
count=date[["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"A14","Survey_Day"]]
count_total=count.groupby(["Cluster_Code",'DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status',"Survey_Day"], as_index=False)["A14"].count()  #this gets all the names and the total surveys submitted for each cluster also as_index=False is extremely important to add
count_male=count[(count['A14']==1)].groupby(["Cluster_Code","A14"])["A14"].count()  # gets male count
#print(count_male)
count_female=count[(count['A14']==2)].groupby(["Cluster_Code","A14"])["A14"].count() # gets female count
#print(count_female)
merge=pd.merge(count_male,count_female, how="outer", on="Cluster_Code") #left_on="Cluster_Code", right_on="Cluster_Code")
#print(merge)
#print(merge.dtypes)
#merge_total=count_total.join(merge)
merge_total=pd.merge(count_total,merge, how="outer", on="Cluster_Code")
#print(count_total)
#print(count.dtypes)
# rename columns
merge_total.rename(columns={"A14":"Total", 
                            "A14_x":"Male",
                            "A14_y":"Female"},
inplace=True, errors="raise")
#changing Nan to 0
merge_total=merge_total.fillna(0)
merge_total



,Cluster_Code,DISTRICT,TEHSIL,RIVER_BASIN,WATER_SHED,UNION_COUNCIL,Sampled_UC_Status,Survey_Day,Total,Male,Female
0,28,Khuzdar,Khuzdar,Hingol,Toba,TOTAK,Actual,2022-12-02,10,9,1.0
1,56,Ziarat,Sinjawi,Nari,Chinjan,POI,Actual,2022-12-03,24,18,6.0
2,57,Ziarat,Ziarat,Nari,Shahrag,ZIARAT,Actual,2022-12-01,1,16,9.0
3,57,Ziarat,Ziarat,Nari,Shahrag,ZIARAT,Actual,2022-12-02,24,16,9.0
4,58,Ziarat,Ziarat,Nari,Shahrag,KWAS,Actual,2022-12-01,23,16,7.0
5,61,Killa Abdullah,Killa Abdullah,Pishin,Killa Abdullah,ARAMBAI,Actual,2022-12-01,13,24,0.0
6,61,Killa Abdullah,Killa Abdullah,Pishin,Killa Abdullah,ARAMBAI,Actual,2022-12-02,11,24,0.0
7,75,Quetta,Quetta Saddar,Pishin,Quetta,BALELI,Actual,2022-12-05,5,4,1.0


In [350]:
#add filter top table 3
filter_table2 = pn.widgets.Tabulator(
    merge_total, pagination='remote', layout='fit_columns', page_size=10, sizing_mode='stretch_width'
    #header_filters=filters , editors=editors
)
filter_table2.add_filter(datetime_range_slider,"Survey_Day")    # first input is the widget and the second input is the column on which the filter is added


In [351]:
#putting it all together
mini_dashboard1=pn.Column(df_google_sheet1,filter_table,datetime_range_slider,filter_table2,file_download_csv)
mini_dashboard1.show()

Launching server at http://localhost:58407


In [ ]:

merge_total.to_csv(r"C:\Users\smhz_\Downloads\data.csv", index=False, header=True)

In [240]:
filter_table3 = pn.widgets.Tabulator(
    date, pagination='remote', layout='fit_columns', page_size=10, sizing_mode='stretch_width'
    #header_filters=filters , editors=editors
)
filter_table3.add_filter(datetime_range_slider,"Survey_Day")

pn.Column(datetime_range_slider,filter_table3)

BokehModel(combine_events=True, render_bundle={'docs_json': {'13be101a-23d8-46d8-a39b-ed292bc16b15': {'defs': …

2022-12-05 18:50:16,047 ERROR: panel.reactive - Callback failed for object named "Datetime Range Slider" changing property {'value': (datetime.datetime(2022, 12, 1, 0, 0),
           datetime.datetime(2022, 12, 1, 1, 55, 20))} 
Traceback (most recent call last):
  File "c:\Users\smhz_\AppData\Local\Programs\Python\Python310\lib\site-packages\panel\reactive.py", line 319, in _process_events
    self.param.update(**self_events)
  File "c:\Users\smhz_\AppData\Local\Programs\Python\Python310\lib\site-packages\param\parameterized.py", line 1898, in update
    self_._batch_call_watchers()
  File "c:\Users\smhz_\AppData\Local\Programs\Python\Python310\lib\site-packages\param\parameterized.py", line 2059, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\smhz_\AppData\Local\Programs\Python\Python310\lib\site-packages\param\parameterized.py", line 2021, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "c:\Users\smhz_\AppData\Local\Programs\Python\Pyt

In [ ]:
# #print(list(useful_data.columns))
# tot_counts=useful_data[['Cluster_Code',"A14"]].value_counts().rename_axis(["Cluster_Code","Gender_Code"]).reset_index(name="count")
# tot_counts
# count=useful_data[['Cluster_Code','DISTRICT','TEHSIL','RIVER_BASIN','WATER_SHED','UNION_COUNCIL','Sampled_UC_Status','A14']]#.groupby(["Cluster_Code","A14"]).count()
# unique_clusters=list(count["Cluster_Code"].unique())
# for cluster in unique_clusters:  # this code will not work if any of the options district, tehsil, rivewr basin, water_shed, union council are not same for a given cluster code
#     c1=count[(count['Cluster_Code']==cluster)].groupby(["A14","DISTRICT","TEHSIL","RIVER_BASIN","WATER_SHED","UNION_COUNCIL","Sampled_UC_Status"]).count()
#     print(c1)


# # d1=c1.
# # print(d1)
# # print(unique_clusters)
# # count_tot=count.groupby(["Cluster_Code","A14","DISTRICT","TEHSIL","RIVER_BASIN","WATER_SHED","UNION_COUNCIL","Sampled_UC_Status"])["A14"].count()
# # count_tot
# count_female
# count_total